### functions

In [ ]:
import pandas as pd
import numpy as np
np.corrcoef([0,1], [0,2])
# import bioformats as bf
import os
import islets
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
from scipy.stats import distributions as dst
plt.rcParams["animation.embed_limit"] = 200
from copy import deepcopy
from scipy.stats import median_abs_deviation

### Import pickle
__NB__: in this notebook you __do not__ merge, discard, or change rois in any way.

In [ ]:
pathToRoi = "/data/Sandra/2020/2020_11_05/Experiment74b.lif_analysis/Series004-16/2021_11_08_9_srdjan_rois.pkl"
regions = islets.load_regions(pathToRoi)

In [ ]:
regions.get_activity([10,100])

In [ ]:
plt.figure()
regions.plotEdges()

In [ ]:
regions.examine()

### Extract events

In [ ]:
rawEvents = islets.EventDistillery.sequential_filtering(regions,)

In [ ]:
ExampleRoi = 187

In [ ]:
singleRaw = rawEvents.query(f"roi=={ExampleRoi}").copy()

In [ ]:
islets.EventDistillery.plot_events(singleRaw,
                                   modify=True # to add color to events
                                  )

In [ ]:
%matplotlib notebook

In [ ]:
distilled, axs, origs = islets.EventDistillery.distill_events_per_roi(singleRaw, regions, plot=True)

This is where you inspect if things make sense. In most cases they should and you need to continue to distilling with default parameters. 

In this case, the default way may not be the best way.

In particular, it looks as like we cannot rely on the `EventDistillery` to catch the big jump(s) around 3300s.
Luckily, by looking at the examiner, it seems that there are not many traces with these jumps.

I will choose to perform default distillation up to a timescale of ~128s, and deal with large events in a different way.

In [ ]:
Events = islets.EventDistillery.distill_events(rawEvents.query("ts<=128"), regions, )

In [ ]:
Events

### Make sure it makes sense

In [ ]:
Events["log10 halfwidth"] = np.log10(Events.halfwidth)
Events["log10 z_max"    ] = np.log10(Events.z_max    )
Events["log10 height"   ] = np.log10(Events.height   )

In [ ]:
Events.columns

In [ ]:
regions.examine_events(Events,y="log10 z_max", x="log10 halfwidth")

In [ ]:
Events = Events.query("z_max>4")

### Save

In [ ]:
for col in Events.columns:
    if "log10" in col:
        del Events[col]

In [ ]:
eventSave = pathToRoi.split("_rois")[0]+"_events.csv"
eventSave

In [ ]:
Events.to_csv(eventSave)